#### Objective
Gather data and create summary table for Biospecimen_Test_Result.xlsx

#### Pre-requisites
The following packages need to be installed in your notebook environment:
1. <a href = https://dev.mysql.com/doc/connector-python/en/>mysql.connector</a>
2. <a href = https://pandas.pydata.org/>pandas</a>
3. <a href = https://numpy.org/>numpy</a>
4. <a href = https://pypi.org/project/tableone/>tableone</a>
5. <a href = https://pypi.org/project/openpyxl/>openpyxl</a>

Be sure to install these packages in one of the directories in the system path(`print sys.path`) for the notebook environment.

You will need access (username and password) to seronet vaccine response database, `seronetdb-Vaccine_Response`, accessible at the AWS endpoint: `seronet-dev-instance.cwe7vdnqsvxr.us-east-1.rds.amazonaws.com`. <br>
**Note:** Store all mysql connector parameters in a `.env` file as shown below. <i>The `.env` needs to be in the same directory as the notebooks</i>.

Ensure that the Excel workbook <i>Release_Data_Dictionary.xlsx</i> is in the same folder as this notebook. This workbook is also on versioned on GitHub.

#### Post-processing files
<u>Summary sheet:</u>
1. Remove all borders from Table 1.
2. Shade table rows in grey/white.
3. Right justify second column.
4. Center all columns reporting values.
5. Autoformat column widths.
6. Highlight all notes in bold.

<u>Detailed report sheets:</u>
1. All columns left justified.
2. Autoformat column widths.
3. Fill all empty cells with "Not Reported".

In [1]:
import mysql.connector as connection
import pandas as pd
import numpy as np
import os
from tableone import TableOne
from openpyxl import load_workbook, Workbook
from openpyxl.styles import Font
import sqlalchemy as sd
import datetime

In [2]:
last_edit = '11/20/2024'

In [3]:
v4_visits = r"C:\Users\breadsp2\Documents\Release_4.0.0\Participant_Visit_Info_4.0.0.xlsx"
v4_visits = pd.read_excel(v4_visits, sheet_name="Detailed_Report")
V4_fitler = False

In [4]:
pd.options.mode.chained_assignment = None  # default='warn'

In [5]:
'''
Create a .env file to store hostname, database, username, and password as environment variables.  
The .env file is stored in the same folder as the notebook.
==============
Format:
HOST=a.b.c.d
DB=<my.database>
USER=john
PWD=abc#123
==============
Note: no space around the "=" sign. Do not put any of the values in quotes.
'''

env = {}
with open("test.env") as f:
    for line in f:
        (k, v) = line.split("=")
        k = k.strip()
        env[k] = v.strip()
        
HOST = env["HOST"]
DB = env["DB"]
USER = env["USER"]
PWD = env["PWD"]

In [6]:
if V4_fitler == True:
    version_num = "4.2.0"
    output_folder = r"C:\Users\breadsp2\Desktop\Release_4.2.0"
else: 
    version_num = "5.2.0"
    output_folder = r"C:\Users\breadsp2\Desktop\Release_5.2.0"
file_sep = os.path.sep
sheet_name = "Biospecimen_Test_Results"
release_date = '2024-01-01'

In [7]:
creds = {'usr': USER, 'pwd': PWD, 'hst': HOST, "prt": 3306, 'dbn': DB}
connstr = "mysql+mysqlconnector://{usr}:{pwd}@{hst}:{prt}/{dbn}"
engine = sd.create_engine(connstr.format(**creds))
conn = engine.connect()

In [8]:
curr_visit_file = output_folder + file_sep + "Participant_Visit_Info_" + version_num + ".xlsx"
curr_visit_data = pd.read_excel(curr_visit_file, sheet_name="Detailed_Report")

In [9]:
#curr_visit_data = curr_visit_data.query("`Visit Purpose` in ['Baseline Visit : Blood Draw','Follow_up Visit: Blood Draw', " + 
#                                        "' Pre_Baseline: Blood Draw']")

In [10]:
demo_file = output_folder + file_sep + "Demographics_" + version_num + ".xlsx"
demo_data = pd.read_excel(demo_file, sheet_name="Detailed_Report")

In [11]:
All_Visits = pd.read_sql(("SELECT p.Seronet_Participant_ID,  nv.Research_Participant_ID, nv.SeroNet_Cohort, nv.Normalized_Visit_Index, " +
                          "nv.`SARS-CoV-2_Vaccine_Type`, nv.Vaccination_Status, nv.Duration_Between_Vaccine_and_Visit, nv.Date_Of_Event, " +
                          "nv.Visit_Info_ID  From Normalized_Visit_Info as nv " + 
                          "join Participant as p on nv.Research_Participant_ID = p.Research_Participant_ID"), conn)

In [12]:
All_Visits = All_Visits.merge(demo_data["Seronet_Participant_ID"])
All_Visits = All_Visits.merge(curr_visit_data[["Seronet_Participant_ID", 'Normalized_Visit_Index']], how="right")

In [13]:
All_Visits.to_csv("All_Visits.csv")

In [14]:
print(len(All_Visits))

15518


In [15]:
bio_test = pd.read_sql(("select * FROM Biospecimen_Test_Results as bt "),conn)

In [16]:
bio_test["Research_Participant_ID"] = [i[:9] for i in bio_test["Visit_Info_ID"]]
bio_test["Harmonized_Interpretation"] = bio_test["Interpretation"]
bio_test["Harmonized_Interpretation"].replace(dict.fromkeys(['Strong Positive', 'Moderate Positive','Weak Positive','Reactive'], "Positive"), inplace=True)
bio_test["Harmonized_Interpretation"].replace(dict.fromkeys(['negative'], "Negative"), inplace=True)

In [17]:
old_col_names = bio_test.columns
old_col_names = old_col_names.to_list()
old_col_names.remove("Research_Participant_ID")

In [18]:
bio_test = bio_test.merge(All_Visits[["Seronet_Participant_ID","Research_Participant_ID", "SeroNet_Cohort",
                                      "Normalized_Visit_Index", "Visit_Info_ID"]], how="outer", indicator="Has_Serology")
bio_test.drop_duplicates(inplace=True)

In [19]:
samp_data = pd.read_sql(("Select * from Biospecimen"), conn)
bio_test = bio_test.merge(samp_data["Visit_Info_ID"], how="outer", indicator="Has Specimen")
bio_test.drop_duplicates(inplace=True)

In [20]:
bio_test = bio_test.merge(All_Visits[["Seronet_Participant_ID", 'Normalized_Visit_Index']])

In [21]:
samp_data = bio_test[["Visit_Info_ID","Has_Serology","Has Specimen"]]
samp_data.drop_duplicates(inplace=True)            
pd.crosstab(samp_data["Has_Serology"], samp_data["Has Specimen"])

Has Specimen,left_only,both
Has_Serology,,
right_only,1065,1737
both,552,12164


In [22]:
bio_test["Serology Status"] = "Missing"

x = bio_test.query("`Has_Serology` == 'both' and `Has Specimen` == 'both'")
bio_test.loc[x.index, "Serology Status"] = "Sample Data Submitted and Has Serology Data"

x = bio_test.query("`Has_Serology` == 'right_only' and `Has Specimen` == 'both'")
bio_test.loc[x.index, "Serology Status"] = "Sample Data Submitted, but no Serology Data"

x = bio_test.query("`Has_Serology` == 'both' and `Has Specimen` == 'left_only'")
bio_test.loc[x.index, "Serology Status"] = "No Sample Data, but Has Serology Data"

x = bio_test.query("`Has_Serology` == 'right_only' and `Has Specimen` == 'left_only'")
bio_test.loc[x.index, "Serology Status"] = "No Sample Data, and No Serology Data"

In [23]:
bio_test["Assay_Target_Organism"].replace("SARS-CoV-2", 'SARS-CoV-2 Virus', inplace=True)
bio_test["Assay_Target"].replace('Spike antigen', "Spike", inplace=True)
bio_test["Assay_Target"].replace('spike protein', "Spike", inplace=True)
bio_test["Assay_Target"].replace('Full Length Spike', "Spike", inplace=True)
bio_test["Assay_Target"].replace('Nucleocapsid (N) antigen', "Nucleocapsid", inplace=True)
bio_test["Assay_Target_Sub_Region"].replace("spike protein receptor-binding domain", "RBD", inplace=True)
bio_test["Assay_Target_Sub_Region"].replace("Nucleocapsid (N) antigen", "N/A", inplace=True)

In [24]:
#col_names = ["Research_Participant_ID", "SeroNet_Cohort", "Normalized_Visit_Index"] + old_col_names
#bio_test = bio_test[col_names]

In [25]:



#negative	54
#same day	7076
#same week	9783
#same month	1235
#over a month	5


In [26]:
col_list = ['Seronet_Participant_ID', 'SeroNet_Cohort', 'Normalized_Visit_Index', 'Visit_Info_ID', "Assay_ID", "Serology Status",
            "Assay_Target", "Measurand_Antibody", "Harmonized_Interpretation", "Biospecimen_Collection_to_Test_Duration"]
visit_data = bio_test[col_list]

In [27]:
visit_data["Total Assays Run"] = 0
x = visit_data.query("Harmonized_Interpretation == 'Positive'")
visit_data.loc[x.index, "Total Assays Run"] = 1 

In [28]:
x = visit_data.groupby("Visit_Info_ID").agg({"Total Assays Run": 'count', "Serology Status": 'max'})
x.reset_index(inplace=True)
#x = x[["Visit_Info_ID", "Total Assays Run", "Serology Status"]]

In [29]:
visit_data = visit_data.query("Assay_ID == Assay_ID")

In [30]:
assay_table = pd.crosstab(visit_data["Visit_Info_ID"],visit_data["Assay_Target"] + [': ']*len(visit_data) + visit_data["Measurand_Antibody"],
                          values = visit_data["Total Assays Run"], aggfunc = sum)
assay_table.reset_index(inplace=True)

#assay_table_2 = pd.crosstab(visit_data["Visit_Info_ID"],visit_data["Assay_ID"],
#                          values = visit_data["Biospecimen_Collection_to_Test_Duration"], aggfunc = min)
#assay_table_2.reset_index(inplace=True)

In [31]:
assay_table.fillna(-1, inplace=True )

In [32]:
assay_table.replace(-1, "Test Not Run", inplace=True)
assay_table.replace(0, "Negative Result", inplace=True)
assay_table.replace(1, "Positive Result", inplace=True)
assay_table.replace(2, "Positive Result", inplace=True)
assay_table.replace(3, "Positive Result", inplace=True)
assay_table.replace(4, "Positive Result", inplace=True)

In [33]:
#assay_table = assay_table.merge(assay_table_2, how="right")
#assay_table = assay_table.merge(All_Visits[["Visit_Info_ID", "SeroNet_Cohort"]], how="right")
assay_table = assay_table.merge(All_Visits, how="right")


assay_table.drop_duplicates(inplace=True)

In [34]:
assay_table = assay_table.merge(x, how="left")

In [35]:
new_cols = ['Seronet_Participant_ID', 'SeroNet_Cohort', 'Normalized_Visit_Index', "Serology Status",
            'Spike: IgM', 'Spike: IgG', 'Spike: Total antibodies',   'Nucleocapsid: Total antibodies', 'Total Assays Run']
assay_table = assay_table[new_cols]

In [36]:
assay_table["Total Assays Run"].fillna(0, inplace=True)
assay_table = assay_table.query("`Total Assays Run` > 0")

In [37]:
bio_test_writer = pd.ExcelWriter(f'{output_folder}{file_sep}{sheet_name}_{version_num}.xlsx')

In [38]:
#if V4_fitler == False:
#    columns = ["Total Assays Run", "Spike: IgM", "Spike: IgG", "Spike: Total antibodies", "Nucleocapsid: IgG", "Nucleocapsid: Total antibodies"]
#    categorical =  ["Total Assays Run", "Spike: IgM", "Spike: IgG", "Spike: Total antibodies", "Nucleocapsid: IgG", "Nucleocapsid: Total antibodies"]

#elif V4_fitler == True:
columns = ["Serology Status", "Total Assays Run", "Spike: IgM", "Spike: IgG", "Spike: Total antibodies", "Nucleocapsid: Total antibodies"]
categorical =  ["Serology Status", "Total Assays Run", "Spike: IgM", "Spike: IgG", "Spike: Total antibodies", "Nucleocapsid: Total antibodies"]

groupby = 'SeroNet_Cohort'

In [39]:
order_dict = {}
order_dict["SeroNet_Cohort"] = ["Healthy Cohort", "Comorbidity Cohort",  "Cancer",   "IBD",  "HIV", "Transplant"]

In [40]:
bio_data_table1 = TableOne(assay_table, columns, categorical, groupby,  order=order_dict, pval=False)

In [41]:
'''
This section of code is not working. Have to solve.
'''
bio_test['Derived_Result'].replace(to_replace = r"NaN", value = "Not Reported", inplace = True)
bio_test['Derived_Result_Units'].replace(to_replace = r"", value = "Not Reported", inplace = True)
bio_test['Raw_Result'].replace(to_replace = r"", value = "Not Reported", inplace = True)
bio_test['Raw_Result_Units'].replace(to_replace = r"", value = "Not Reported", inplace = True)
bio_test['Positive_Control_Reading'].replace(to_replace = r"", value = "Not Reported", inplace = True)
bio_test['Negative_Control_Reading'].replace(to_replace = r"", value = "Not Reported", inplace = True)

In [42]:
bio_test.sort_values(["Seronet_Participant_ID", "Normalized_Visit_Index"], inplace=True)

In [43]:
bio_test["Collection_To_Test_Interval"] = "Unknown"

x = bio_test.query("Biospecimen_Collection_to_Test_Duration < 0")
bio_test.loc[x.index, "Collection_To_Test_Interval"] = "Negative Test Duration"

x = bio_test.query("Biospecimen_Collection_to_Test_Duration >= 0 and Biospecimen_Collection_to_Test_Duration <= 24")
bio_test.loc[x.index, "Collection_To_Test_Interval"] = "Tested Same Day (0 to 24 hours)"

x = bio_test.query("Biospecimen_Collection_to_Test_Duration > 24 and Biospecimen_Collection_to_Test_Duration <= 48")
bio_test.loc[x.index, "Collection_To_Test_Interval"] = "Tested Next  Day (24 to 48 hours)"

x = bio_test.query("Biospecimen_Collection_to_Test_Duration > 48 and Biospecimen_Collection_to_Test_Duration <= 168")
bio_test.loc[x.index, "Collection_To_Test_Interval"] = "Tested with in a week (3 to 7 days)"

x = bio_test.query("Biospecimen_Collection_to_Test_Duration > 168")
bio_test.loc[x.index, "Collection_To_Test_Interval"] = "Tested after week (over 7 days)"

In [44]:
assay_convert = pd.read_sql(("SELECT * FROM `seronetdb-Vaccine_Response_v2`.Assay_ID_Convernt;"), conn)
bio_test = bio_test.merge(assay_convert)

In [45]:
bio_test.columns

Index(['Visit_Info_ID', 'Assay_ID', 'Instrument_ID', 'Test_Operator_Initials',
       'Assay_Kit_Lot_Number', 'Test_Batch_ID',
       'Biospecimen_Collection_to_Test_Duration', 'Assay_Target_Organism',
       'Assay_Target', 'Assay_Target_Sub_Region', 'Measurand_Antibody',
       'Assay_Replicate', 'Sample_Type', 'Sample_Dilution', 'Interpretation',
       'Derived_Result', 'Derived_Result_Units', 'Raw_Result',
       'Raw_Result_Units', 'Positive_Control_Reading',
       'Negative_Control_Reading', 'Biospecimen_Test_Results_Comments',
       'Research_Participant_ID', 'Harmonized_Interpretation',
       'Seronet_Participant_ID', 'SeroNet_Cohort', 'Normalized_Visit_Index',
       'Has_Serology', 'Has Specimen', 'Serology Status',
       'Collection_To_Test_Interval', 'Seronet_Assay_ID'],
      dtype='object')

In [46]:
new_cols = ["Seronet_Participant_ID", "SeroNet_Cohort", "Normalized_Visit_Index", "Instrument_ID", 
            "Assay_Kit_Lot_Number", "Test_Batch_ID", "Biospecimen_Collection_to_Test_Duration", "Collection_To_Test_Interval",
            "Seronet_Assay_ID",
            "Assay_Target_Organism", "Assay_Target", "Assay_Target_Sub_Region", "Measurand_Antibody", "Assay_Replicate", 
            "Sample_Type", "Sample_Dilution", "Interpretation", "Harmonized_Interpretation" ,"Derived_Result", 
            "Derived_Result_Units", "Raw_Result", "Raw_Result_Units", "Positive_Control_Reading", "Negative_Control_Reading",
            "Biospecimen_Test_Results_Comments"]
bio_test  = bio_test[new_cols]

In [47]:
#bio_test = bio_test.drop(["Assay_ID", "Visit_Info_ID", "Research_Participant_ID"], axis=1)

In [48]:
columns = ["Seronet_Assay_ID","Collection_To_Test_Interval"]
categorical =  ["Seronet_Assay_ID","Collection_To_Test_Interval"]
groupby = 'SeroNet_Cohort'

In [49]:
order_dict = {}
order_dict["Seronet_Assay_ID"] = list(set(bio_test["Seronet_Assay_ID"]))
order_dict["Seronet_Assay_ID"].sort()
order_dict["SeroNet_Cohort"] = ["Healthy Cohort", "Comorbidity Cohort",  "Cancer",   "IBD",  "HIV", "Transplant"]

order_dict["Collection_To_Test_Interval"] = ['Negative Test Duration', 'Tested Same Day (0 to 24 hours)', 'Tested Next  Day (24 to 48 hours)',
                                            'Tested with in a week (3 to 7 days)', 'Tested after week (over 7 days)']

In [50]:
bio_test["Positive_Control_Reading"].fillna("Not Reported", inplace=True)
bio_test["Negative_Control_Reading"].fillna("Not Reported", inplace=True)
bio_test["Biospecimen_Test_Results_Comments"].fillna("N/A", inplace=True)
bio_test["Biospecimen_Test_Results_Comments"] = bio_test["Biospecimen_Test_Results_Comments"].replace("", "N/A")

In [51]:
bio_test = bio_test.query("Biospecimen_Collection_to_Test_Duration >= 0")

In [52]:
bio_data_table2 = TableOne(bio_test, columns, categorical, groupby, order=order_dict)

C:\ProgramData\anaconda3\Lib\site-packages\tableone\tableone.py:1457: UserWarning: Order value not found: Collection_To_Test_Interval: ['Negative Test Duration']
  warnings.warn(("Order value not found: "


In [53]:
bio_data_table1.to_excel(bio_test_writer, 'Assays_Results_Summary')
assay_table.to_excel(bio_test_writer, "Assays by Visit", index = False)
bio_data_table2.to_excel(bio_test_writer, "Individal Assay Summary")
bio_test.to_excel(bio_test_writer, "Test Result Data", index = False)

In [54]:
workbook = bio_test_writer.book
worksheet = bio_test_writer.sheets['Assays_Results_Summary']
column_fmt = workbook.add_format({'font_name': 'Arial', 'font_size': 10, 'align': 'center'})
worksheet.set_column(0, 0, 35, column_fmt)   #this is column header names
worksheet.set_column(1, 1, 45, column_fmt)   #this is sub column header names
worksheet.set_column(2, 3, 12, column_fmt)   #all the data
worksheet.set_column(4, 17, 20, column_fmt)  #all the data

row_fmt = workbook.add_format({'text_wrap': True})
worksheet.set_row(1, 30, row_fmt)

0

In [55]:
workbook = bio_test_writer.book
worksheet = bio_test_writer.sheets['Individal Assay Summary']
column_fmt = workbook.add_format({'font_name': 'Arial', 'font_size': 10, 'align': 'center'})
worksheet.set_column(0, 0, 35, column_fmt)   #this is column header names
worksheet.set_column(1, 1, 35, column_fmt)   #this is sub column header names
worksheet.set_column(2, 3, 12, column_fmt)   #all the data
worksheet.set_column(4, 17, 20, column_fmt)  #all the data

row_fmt = workbook.add_format({'text_wrap': True})
worksheet.set_row(1, 30, row_fmt)

0

In [56]:
#biospecimen_test_results = bio_test.merge(assay_summary[["Visit_Info_ID", "Normalized_Visit_Index", 'Seronet_Cohort']])

In [57]:
'''
Ensure that the Release_Data_Dictionary.xlsx workbook is stored in the same folder as this notebook.
'''
dictionary = pd.read_excel("Release_Data_Dictionary_External.xlsx", sheet_name = "Testing_Dictionary")

In [58]:
dictionary.to_excel(bio_test_writer, 'Data_Dictionary', index = False)

In [59]:
bio_test_writer.close()

In [60]:
'''
Open the workbook to add additional notes to the Summary page.
Each note shoud be on a separate line. Add note as a single value array: noteX = ['noteX: This is a sample note.']
Append note to sheet.
Save workbook.
'''
wb = load_workbook(f'{output_folder}{file_sep}{sheet_name}_{version_num}.xlsx')
sheet = wb['Assays_Results_Summary']
note0 = ['']
note1 = ['Note1: There are 2,289 visits not shown']
note2 = ['                 There are 1,737 visits with samples, but no serology data was submitted']
note3 = ['                 There are 1,065 visits with no samples and no serology data was submitted']
sheet.append(note0)
sheet.append(note1)
sheet.append(note2)
sheet.append(note3)
wb.save(filename = f'{output_folder}{file_sep}{sheet_name}_{version_num}.xlsx')